<h2 style="color:blue" align="center">Deep Neural Network (DNN) on MNIST</h2>

#### Import the necessary libraries

In [110]:
import tensorflow as tf 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time, math 

from tensorflow import keras 
from keras.models import Sequential
from keras.layers import Dense 
from keras.callbacks import Callback, CSVLogger 

#### Load and split the dataset into training and testing set

MNIST - Handwritten digits recognition

In [111]:
#(X_train,y_train),(X_test,y_test) = keras.datasets.mnist.load_data()

MNIST - Fashion dataset

In [112]:
(X_train,y_train),(X_test,y_test) = keras.datasets.fashion_mnist.load_data()

#### Flattening

In [113]:
X_train_flattened = X_train.reshape(len(X_train), 28 * 28)
X_test_flattened = X_test.reshape(len(X_test),  28 * 28) 

#### Normalize the input dataset

In [114]:
X_train_normalized = X_train_flattened / 255
X_test_normalized = X_test_flattened / 255

#### Define DNN model with two layers, optimizer, metrics, and loss function

In [115]:
np.random.seed(3)

def get_model(): 
    model = Sequential([
        # input layer 784 neurons to first hidden layer with 64 neurons
        Dense(64, input_shape = (784,), activation='relu'), 
        # first hidden layer to second hidden layer
        Dense(64, activation='relu'),  
        # Output layer with 10 neurons
        Dense(10, activation='softmax')
    ])
    
    # General SGD
    #opti = keras.optimizers.SGD(learning_rate=0.01)
    
    # SGD with momentum
    #opti = keras.optimizers.SGD(learning_rate=0.01, momentum=0.6)
    
    # SGD with Nesterov momentum 
    #opti = keras.optimizers.SGD(learning_rate=0.01, momentum=0.6, nesterov=True)
    
    # RMSprop 
    #opti = keras.optimizers.RMSprop(learning_rate=0.001, momentum=0.6)
    
    # Adam
    opti = keras.optimizers.Adam(learning_rate=0.001) 
    
    # Adamax
    #opti = keras.optimizers.Adamax(learning_rate=0.001) 
    
    model.compile(
        optimizer = opti,
        loss = 'sparse_categorical_crossentropy',
        metrics = ['accuracy']
    )
    
    return model

#### Custom callbacks

For generic optimizer model

In [116]:
class CustomCallbackGeneric(Callback):  
    # Training stop criteria
    stop_at = 0.99
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')> self.stop_at):  
            self.model.stop_training = True

For HM based optimizer model

In [117]:
class CustomCallbackHM(Callback):  
    # Stop the algorithm when the following accuracy reached 
    stop_at = 0.995  
    
    initial_weights = 0
    previous_weights = 0
    call_hm = 0 
     
    def on_train_begin(self, logs=None):
        self.initial_weights = model_hm.get_weights() 
        self.initial_weights = np.array(self.initial_weights,dtype=object)
        self.previous_weights = self.initial_weights
        # Harmonic mean based weights calculation
        self.call_hm = np.vectorize(self.apply_hm)  

    def on_epoch_end(self, epoch, logs={}):
        # Training stop criteria
        if(logs.get('accuracy')> self.stop_at):  
            self.model.stop_training = True
        
        counter = 0
        num_layers = len(model_hm.layers)  
        current_weights = model_hm.get_weights()
        current_weights = np.array(current_weights,dtype=object)        

        for i in range(num_layers):  
            # Harmonic mean based weights calculation
            current_weights[counter] = self.call_hm(self.previous_weights[counter], current_weights[counter])
            counter = counter + 2
            
        # Updating the model with new weights
        updated = current_weights.tolist()   
        model_hm.set_weights(updated)
        self.previous_weights = current_weights
        
    def apply_hm(self, v1, v2):     
        if v1==0 or v2==0:
            return v2
        elif v1>0 and v2>0:
            hm = 2*v1*v2/(v1+v2)
            min1 = min(v1,v2)
            diff = abs(hm-min1)
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        elif v1<0 and v2<0:
            hm = 2*v1*v2/(v1+v2)
            max1 = max(v1,v2)
            diff = abs(hm-max1)
            if v2 > v1:
                return v2 + diff
            else:
                return v2 - diff
        else:
            return v2  

To record loss and accuracy in CSV file

In [118]:
logger_generic_model = CSVLogger('Generic_model_MNIST.csv', append=False, separator=',')
logger_hm_model = CSVLogger('HM_model_MNIST.csv', append=False, separator=',')

#### Training

Generic optimizer model

In [119]:
model_wihtout_hm = get_model() 
st = time.time()
model_wihtout_hm.fit(X_train_normalized, y_train, epochs = 135, verbose=1, callbacks=[CustomCallbackGeneric(), logger_generic_model], batch_size = X_train.shape[0]) 
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Epoch 1/135
1/1 [==============================] - 1s 722ms/step - loss: 2.3702 - accuracy: 0.1018
Epoch 2/135
1/1 [==============================] - 0s 148ms/step - loss: 2.2419 - accuracy: 0.1424
Epoch 3/135
1/1 [==============================] - 0s 143ms/step - loss: 2.1498 - accuracy: 0.2135
Epoch 4/135
1/1 [==============================] - 0s 149ms/step - loss: 2.0718 - accuracy: 0.2849
Epoch 5/135
1/1 [==============================] - 0s 172ms/step - loss: 1.9945 - accuracy: 0.3627
Epoch 6/135
1/1 [==============================] - 0s 163ms/step - loss: 1.9146 - accuracy: 0.4346
Epoch 7/135
1/1 [==============================] - 0s 155ms/step - loss: 1.8331 - accuracy: 0.4964
Epoch 8/135
1/1 [==============================] - 0s 149ms/step - loss: 1.7523 - accuracy: 0.5329
Epoch 9/135
1/1 [==============================] - 0s 164ms/step - loss: 1.6711 - accuracy: 0.5582
Epoch 10/135
1/1 [==============================] - 0s 149ms/step - loss: 1.5898 - accuracy: 0.5765
Epoch 11/

1/1 [==============================] - 0s 161ms/step - loss: 0.4704 - accuracy: 0.8392
Epoch 84/135
1/1 [==============================] - 0s 144ms/step - loss: 0.4697 - accuracy: 0.8393
Epoch 85/135
1/1 [==============================] - 0s 142ms/step - loss: 0.4675 - accuracy: 0.8404
Epoch 86/135
1/1 [==============================] - 0s 150ms/step - loss: 0.4652 - accuracy: 0.8408
Epoch 87/135
1/1 [==============================] - 0s 157ms/step - loss: 0.4643 - accuracy: 0.8415
Epoch 88/135
1/1 [==============================] - 0s 162ms/step - loss: 0.4626 - accuracy: 0.8421
Epoch 89/135
1/1 [==============================] - 0s 159ms/step - loss: 0.4605 - accuracy: 0.8427
Epoch 90/135
1/1 [==============================] - 0s 159ms/step - loss: 0.4593 - accuracy: 0.8431
Epoch 91/135
1/1 [==============================] - 0s 160ms/step - loss: 0.4580 - accuracy: 0.8437
Epoch 92/135
1/1 [==============================] - 0s 148ms/step - loss: 0.4561 - accuracy: 0.8443
Epoch 93/135


HM based optimizer model

In [121]:
model_hm = get_model() 
st = time.time()
model_hm.fit(X_train_normalized, y_train, epochs = 94, verbose=1, callbacks=[CustomCallbackHM(),logger_hm_model], batch_size = X_train.shape[0]) 
et = time.time()
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

Epoch 1/94
1/1 [==============================] - 1s 724ms/step - loss: 2.3155 - accuracy: 0.1137
Epoch 2/94
1/1 [==============================] - 0s 207ms/step - loss: 2.1272 - accuracy: 0.2892
Epoch 3/94
1/1 [==============================] - 0s 200ms/step - loss: 1.9921 - accuracy: 0.3880
Epoch 4/94
1/1 [==============================] - 0s 200ms/step - loss: 1.8600 - accuracy: 0.4850
Epoch 5/94
1/1 [==============================] - 0s 188ms/step - loss: 1.7271 - accuracy: 0.5570
Epoch 6/94
1/1 [==============================] - 0s 210ms/step - loss: 1.5969 - accuracy: 0.5748
Epoch 7/94
1/1 [==============================] - 0s 204ms/step - loss: 1.4724 - accuracy: 0.5831
Epoch 8/94
1/1 [==============================] - 0s 187ms/step - loss: 1.3568 - accuracy: 0.5985
Epoch 9/94
1/1 [==============================] - 0s 192ms/step - loss: 1.2530 - accuracy: 0.6238
Epoch 10/94
1/1 [==============================] - 0s 212ms/step - loss: 1.1618 - accuracy: 0.6554
Epoch 11/94
1/1 [==

Model summary

In [122]:
model_hm.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_69 (Dense)            (None, 64)                50240     
                                                                 
 dense_70 (Dense)            (None, 64)                4160      
                                                                 
 dense_71 (Dense)            (None, 10)                650       
                                                                 
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


###### Testing the model

Generic opimizer model

In [123]:
model_wihtout_hm.evaluate(X_test_normalized, y_test)

313/313 [==============================] - 0s 998us/step - loss: 0.4487 - accuracy: 0.8421


[0.4487186670303345, 0.8421000242233276]

HM based optimizer model

In [124]:
model_hm.evaluate(X_test_normalized, y_test)

313/313 [==============================] - 1s 1ms/step - loss: 0.4434 - accuracy: 0.8449


[0.4433857798576355, 0.8449000120162964]